<a href="https://colab.research.google.com/github/TungAnhDep/Fundamental-Deep-Learning/blob/main/Deep_CNN_AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms

from torchsummary import summary

In [ ]:
class AlexNet(nn.Module):
  def __init__(self, num_class):
    super(AlexNet, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(3, 96, kernel_size = 11, stride = 4, padding = 0),
        nn.BatchNorm2d(96),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 3, stride = 2)
    )
    self.layer2= nn.Sequential(
        nn.Conv2d(96, 256, kernel_size = 5, stride = 1, padding =2),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size =3, stride =2)
    )
    self.layer3 = nn.Sequential(
        nn.Conv2d(256, 384, kernel_size =3, stride =1, padding =1),
        nn.BatchNorm2d(384),
        nn.ReLU()
    )
    #Tai sao khong dung MaxPooling
    self.layer4 = nn.Sequential(
        nn.Conv2d(384, 384, kernel_size =3, stride =1, padding =1),
        nn.BatchNorm2d(384),
        nn.ReLU()
    )
    self.layer5 = nn.Sequential(
        nn.Conv2d(384, 256, kernel_size = 3, stride = 1, padding =1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 3, stride = 2)
    )
    self.linear1 = nn.Linear(256*5*5, 4096)
    self.linear2 = nn.Linear(4096, 4096)
    self.linear3 = nn.Linear(4096, num_class)
    self.dropout = nn.Dropout(0.5)
    self.flatten = nn.Flatten()
    self.relu = nn.ReLU()
  def forward(self,x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = self.layer5(out)
    out = self.flatten(out)
    out = self.linear1(out)
    out = self.relu(out)
    out = self.dropout(out)
    out = self.linear2(out)
    out = self.relu(out)
    out = self.dropout(out)
    out = self.linear3(out)
    return out

In [ ]:
train_dataset = datasets.CIFAR10(root = '.', train = True, download = True, transform = transforms.Compose(
    [
      transforms.Resize((224, 224)),

      transforms.ToTensor(),

    ]
))
valid_dataset = datasets.CIFAR10(root = '.', train = False, download = True, transform = transforms.Compose([
    transforms.Resize((224,224)),

    transforms.ToTensor(),

]))
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
valid_loader = DataLoader(valid_dataset, batch_size = 64)


100%|██████████| 170M/170M [00:13<00:00, 12.7MB/s]


In [ ]:
train_N = len(train_loader.dataset)
valid_N = len(valid_loader.dataset)
train_N

50000

In [ ]:
train_dataset[0][0].shape

torch.Size([3, 224, 224])

In [ ]:
train_dataset[0][0]

tensor([[[0.2314, 0.2314, 0.2314,  ..., 0.5804, 0.5804, 0.5804],
         [0.2314, 0.2314, 0.2314,  ..., 0.5804, 0.5804, 0.5804],
         [0.2314, 0.2314, 0.2314,  ..., 0.5804, 0.5804, 0.5804],
         ...,
         [0.6941, 0.6941, 0.6941,  ..., 0.4824, 0.4824, 0.4824],
         [0.6941, 0.6941, 0.6941,  ..., 0.4824, 0.4824, 0.4824],
         [0.6941, 0.6941, 0.6941,  ..., 0.4824, 0.4824, 0.4824]],

        [[0.2431, 0.2431, 0.2431,  ..., 0.4863, 0.4863, 0.4863],
         [0.2431, 0.2431, 0.2431,  ..., 0.4863, 0.4863, 0.4863],
         [0.2431, 0.2431, 0.2431,  ..., 0.4863, 0.4863, 0.4863],
         ...,
         [0.5647, 0.5647, 0.5647,  ..., 0.3608, 0.3608, 0.3608],
         [0.5647, 0.5647, 0.5647,  ..., 0.3608, 0.3608, 0.3608],
         [0.5647, 0.5647, 0.5647,  ..., 0.3608, 0.3608, 0.3608]],

        [[0.2471, 0.2471, 0.2471,  ..., 0.4039, 0.4039, 0.4039],
         [0.2471, 0.2471, 0.2471,  ..., 0.4039, 0.4039, 0.4039],
         [0.2471, 0.2471, 0.2471,  ..., 0.4039, 0.4039, 0.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
model = AlexNet(num_class = 10).to(device)
summary(model, (3, 224, 224))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 54, 54]          34,944
       BatchNorm2d-2           [-1, 96, 54, 54]             192
              ReLU-3           [-1, 96, 54, 54]               0
         MaxPool2d-4           [-1, 96, 26, 26]               0
            Conv2d-5          [-1, 256, 26, 26]         614,656
       BatchNorm2d-6          [-1, 256, 26, 26]             512
              ReLU-7          [-1, 256, 26, 26]               0
         MaxPool2d-8          [-1, 256, 12, 12]               0
            Conv2d-9          [-1, 384, 12, 12]         885,120
      BatchNorm2d-10          [-1, 384, 12, 12]             768
             ReLU-11          [-1, 384, 12, 12]               0
           Conv2d-12          [-1, 384, 12, 12]       1,327,488
      BatchNorm2d-13          [-1, 384, 12, 12]             768
             ReLU-14          [-1,

In [ ]:
model = torch.compile(model.to(device))
model

OptimizedModule(
  (_orig_mod): AlexNet(
    (layer1): Sequential(
      (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
      (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (layer2): Sequential(
      (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (layer3): Sequential(
      (0): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (layer4): Sequential(
      (0): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(384, eps=1e-05,

In [ ]:
optimizer = Adam(model.parameters(), lr = 1e-3)
loss_function = nn.CrossEntropyLoss()

In [ ]:
def get_batch_accuracy(output, y, N):
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

In [ ]:
def train():
  loss = 0
  accuracy = 0
  model.train()
  for x,y in train_loader:
    x,y = x.to(device), y.to(device)
    output = model(x)
    optimizer.zero_grad()
    batch_loss= loss_function(output, y)
    batch_loss.backward()
    optimizer.step()
    loss+=batch_loss.item()
    accuracy+=get_batch_accuracy(output, y, train_N)
  print('Training Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))


In [ ]:
def evaluate():
  loss = 0
  accuracy = 0
  model.eval()
  with torch.no_grad():
    for x,y in valid_loader:
      x,y = x.to(device), y.to(device)
      output = model(x)
      batch_loss = loss_function(output,y)
      loss+=batch_loss.item()
      accuracy+=get_batch_accuracy(output,y,valid_N)
    print('Evaluation Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [ ]:
epochs = 10
for epoch in range (epochs):
  print('Epoch: {}'.format(epoch+1))
  train()
  evaluate()

Epoch: 0


W0426 01:41:45.406000 251 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode


Training Loss: 1482.1797 Accuracy: 0.3232
Evaluation Loss: 256.6658 Accuracy: 0.4195
Epoch: 1
Training Loss: 1119.9465 Accuracy: 0.4793
Evaluation Loss: 219.0167 Accuracy: 0.4993
Epoch: 2
Training Loss: 930.9122 Accuracy: 0.5791
Evaluation Loss: 197.8422 Accuracy: 0.5579
Epoch: 3
Training Loss: 796.6260 Accuracy: 0.6453
Evaluation Loss: 160.2325 Accuracy: 0.6479
Epoch: 4
Training Loss: 686.7044 Accuracy: 0.6993
Evaluation Loss: 154.5816 Accuracy: 0.6669
Epoch: 5
Training Loss: 602.1230 Accuracy: 0.7374
Evaluation Loss: 117.6051 Accuracy: 0.7523
Epoch: 6
Training Loss: 524.2050 Accuracy: 0.7726
Evaluation Loss: 117.6440 Accuracy: 0.7438
Epoch: 7
Training Loss: 465.1837 Accuracy: 0.7990
Evaluation Loss: 136.5983 Accuracy: 0.7135
Epoch: 8
Training Loss: 415.6342 Accuracy: 0.8208
Evaluation Loss: 103.9239 Accuracy: 0.7780
Epoch: 9
Training Loss: 370.4234 Accuracy: 0.8426
Evaluation Loss: 108.8880 Accuracy: 0.7685


In [ ]:
prediction = model(valid_dataset[0][0].unsqueeze(0).to(device))
prediction.argmax()

tensor(3, device='cuda:0')

In [ ]:
valid_dataset[0][1]

3